# Extract the dominant colors of skin, eyes and hair

In [2]:
import cv2
import numpy as np
import dlib
import Queue
from collections import OrderedDict
from IPython.core.debugger import Tracer

In [3]:
FACIAL_LANDMARKS_IDXS = OrderedDict([
    #("mouth", (48, 68)),
    ("right_eyebrow", (17, 22)), # hair
    #("left_eyebrow", (22, 27)),
    ("right_eye", (36, 42)), # eye
    #("left_eye", (42, 48)),
    ("nose", (27, 35)), # skin
    #("jaw", (0, 17))
])

def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)

def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)

    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

# Image is already a face no need to extract the face
def extract_face_parts(image, predictor_path):
    predictor = dlib.shape_predictor(predictor_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rect = dlib.rectangle(left=0, top=0, right=image.shape[1], bottom=image.shape[0])
    shape = predictor(gray, rect)
    shape = shape_to_np(shape)
    
    rois = []

    for (name, (i, j)) in FACIAL_LANDMARKS_IDXS.items():
        (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
        rois.append((x, y, w, h))
    
    return rois

In [19]:
class TreeColorNode(object):
    
    def __init__(self, class_id):
        self.class_id = class_id
        
        self.mean = None
        self.cov = None
        
        self.left = None
        self.right = None

def get_leaves(root):
    #print("get_leaves")
    ret = []
    queue = Queue.Queue()
    queue.put(root)
    
    while queue.qsize() > 0:
        current = queue.get()
        
        if current.left != None and current.right != None:
            queue.put(current.left)
            queue.put(current.right)
            continue
            
        ret.append(current)
        
    return ret

def get_dominant_colors(root):
    #print("get_dominant_color")
    leaves = get_leaves(root)
    ret = []
    
    for leaf in leaves:
        mean = leaf.mean * 255.0 # denormalize
        ret.append(mean)
        
    return ret

def get_next_classid(root):
    #print("get_next_classid")
    max_id = 0
    queue = Queue.Queue()
    queue.put(root)
    
    while queue.qsize() > 0:
        current = queue.get()
    
        if current.class_id > max_id:
            max_id = current.class_id

        if current.left != None:
            queue.put(current.left)

        if current.right != None:
            queue.put(current.right)
            
    return max_id + 1

def get_class_mean_cov(image, classes, node):
    #print("get_class_mean_cov")
    assert image.shape[0] == classes.shape[0]
    assert image.shape[1] == classes.shape[1]
    
    height, width, depth = image.shape
    class_id = node.class_id
    
    cov = np.zeros((3, 3))
    
    is_class = classes == class_id # boolean array to identify class_id pixels
    class_pixels = image[is_class] / 255.0 # normalize

    mean = class_pixels.mean(axis=0).reshape(3, 1)
    sum_ = class_pixels.sum(axis=0).reshape(3, 1)

    shape = class_pixels.shape
    pixcount = shape[0]

    for pixel in class_pixels:
        p = pixel.reshape(3, 1)
        cov = cov + p * p.T

    cov = cov - (sum_ * sum_.T) / (pixcount)
    
    node.mean = mean
    node.cov = cov
    
def partition_class(image, classes, next_id, node):
    #print("partition_class")
    assert image.shape[0] == classes.shape[0]
    assert image.shape[1] == classes.shape[1]
    
    height, width, depth = image.shape
    class_id = node.class_id
    
    new_id_left = next_id
    new_id_right = next_id + 1
    
    mean = node.mean
    cov = node.cov
    
    shape = cov.shape
    eigenvectors = np.zeros(shape)
    eigenvalues = np.zeros(shape[0])
    cv2.eigen(cov, eigenvalues, eigenvectors)

    # print("\neigenvalues")
    # print(eigenvalues)

    # print("\neigenvectors")
    # print(eigenvectors)
    
    eig = eigenvectors[0]
    
    comparison_value = eig * mean
    # print("\ncomparison_value")
    # print(comparison_value)
    
    node.left = TreeColorNode(new_id_left)
    node.right = TreeColorNode(new_id_right)
    
    for y in range(height):
        for x in range(width):
            if classes[y, x] != class_id:
                continue
                
            pixel = image[y, x, :].reshape(3, 1)
            scaled = pixel / 255.0

            this_value = eig * scaled
            #print("\nthis_value")
            #print(this_value)
            if(this_value[0, 0] <= comparison_value[0, 0]):
                classes[y, x] = new_id_left
            else:
                classes[y, x] = new_id_right      
                
def get_max_eigenvalue_node(current):
    #print("get_max_eigenvalue_node")
    max_eigen = -1
    queue = Queue.Queue()
    queue.put(current)
    
    ret = current
    if(current.left == None and current == None):
        return current
    
    while queue.qsize() > 0:
        node = queue.get()
        if(node.left != None and node != None):
            queue.put(node.left)
            queue.put(node.right)
            continue
            
        shape = node.cov.shape
        eigenvectors = np.zeros(shape)
        eigenvalues = np.zeros(shape[0])
        cv2.eigen(node.cov, eigenvalues, eigenvectors)
        val = eigenvalues[0]
        if(val > max_eigen):
            max_eigen = val
            ret = node
            
    return ret


def find_dominant_colors(image, count=3):
    #print("find_dominant_colors")
    height, width, depth = image.shape
    
    classes = np.ones((height, width)) # will store the class of each pixel
    
    root_ = TreeColorNode(1)
    next_ = root_
    
    get_class_mean_cov(image, classes, root_)
    
    for c in range(count):
        next_ = get_max_eigenvalue_node(root_)
        next_id = get_next_classid(root_)
        partition_class(image, classes, next_id, next_)
        get_class_mean_cov(image, classes, next_.left)
        get_class_mean_cov(image, classes, next_.right)
        
    colors = get_dominant_colors(root_)
    
#     quantized = get_quantized_image(classes, root)
#     viewable = get_viewable_image(classes)
#     dom = get_dominant_palette(colors)
    
    # create visualizations
    
    return colors

In [22]:
def try_it(face_path):
    image = cv2.imread(face_path)
    image_f = image.astype(float)
    rois = extract_face_parts(image, "shape_predictor_68_face_landmarks.dat")
    
    for roi in rois: # eyebrow, eye, nose
        (x, y, w, h) = roi
        face_part = image_f[y:y+h, x:x+w]
        colors = find_dominant_colors(face_part, 3)
        
        print("face_part")
        for c in colors:
            print(c.squeeze())   

try_it("test_face.ppm")
       

face_part
[ 31.17054264  35.03100775  36.48837209]
[ 35.83505155  43.63917526  48.        ]
[ 43.26732673  62.56435644  80.43564356]
[  55.17241379   84.63793103  109.86206897]
face_part
[ 30.11363636  36.25        41.06818182]
[ 37.96296296  48.81481481  59.66666667]
[ 48.85294118  67.61764706  84.61764706]
[  64.25         90.67857143  108.39285714]
face_part
[ 36.56779661  49.34745763  64.95762712]
[  50.26415094   80.18238994  116.71698113]
[  62.94039735  108.64900662  156.50331126]
[  75.42424242  129.31060606  179.03787879]


In [5]:
-

(128, 128, 3)


NameError: global name 'width' is not defined